In [1]:
#part 1

In [50]:
import pandas as pd
import numpy as np

# Load your data for the 'colleges' DataFrame
colleges = pd.read_csv('/Users/jgmac22/Downloads/colleges.csv', low_memory=False)

# Filter colleges with undergraduate populations greater than 700
colleges = colleges[colleges['UGDS'] > 700]

# Replace 'PrivacySuppressed' with NaN and convert to numeric for several columns
columns_to_convert = ['ADM_RATE_SUPP', 'C150_4_POOLED_SUPP', 'MN_EARN_WNE_P10', 'RET_FT4_POOLED_SUPP', ]
for col in columns_to_convert:
    colleges[col] = colleges[col].replace('PS', np.nan)
    colleges[col] = pd.to_numeric(colleges[col], errors='coerce')

# Drop rows where any of the key metrics is NaN
colleges = colleges.dropna(subset=columns_to_convert)

# Rankings based on admission rate, graduation rate, median earnings, student-to-faculty ratio, retention rate, and SAT scores
colleges['Selectivity Rank'] = colleges['ADM_RATE_SUPP'].rank(method='min', ascending=True)
colleges['Graduation Rate Rank'] = colleges['C150_4_POOLED_SUPP'].rank(method='min', ascending=False)
colleges['Earnings Rank'] = colleges['MN_EARN_WNE_P10'].rank(method='min', ascending=False)
colleges['Retention Rate Rank'] = colleges['RET_FT4_POOLED_SUPP'].rank(method='min', ascending=False)

# Select necessary columns and calculate a combined rank (example: average of ranks)
colleges['Combined Rank'] = colleges[['Selectivity Rank', 'Graduation Rate Rank', 'Earnings Rank', 'Retention Rate Rank']].mean(axis=1)

# Sort colleges by combined rank
ranked_colleges = colleges[['INSTNM', 'ADM_RATE_SUPP', 'C150_4_POOLED_SUPP', 'MN_EARN_WNE_P10', 'RET_FT4_POOLED_SUPP', 'Selectivity Rank', 'Graduation Rate Rank', 'Earnings Rank',  'Retention Rate Rank',  'Combined Rank']]
ranked_colleges_sorted = ranked_colleges.sort_values(by='Combined Rank')
ranked_colleges_sorted.reset_index(drop=True, inplace=True)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

print(ranked_colleges_sorted.head(400))  # Shows the top 50 entries based on the combined rank


                                                INSTNM  ADM_RATE_SUPP  \
0                Massachusetts Institute of Technology         0.0396   
1                                      Yale University         0.0457   
2                                  Stanford University         0.0368   
3                                   Harvard University         0.0324   
4                           University of Pennsylvania         0.0650   
5          Columbia University in the City of New York         0.0395   
6                                University of Chicago         0.0543   
7                                      Duke University         0.0635   
8                   California Institute of Technology         0.0269   
9                                 Princeton University         0.0570   
10                                   Dartmouth College         0.0638   
11                                    Brown University         0.0506   
12                             Northwestern Univers

This methodology fairly accurately captures the rankings seen across the various ranking websites by professional news organizations. The use of admission rate, graduation rate, and mean earnings 10 years after graduation leads to similar rankings as seen online. Although thee are important criteria for a institution, it leads to issues when valuing these scores over others and using previous ranksings as a baseline for the new ones. Schools are sort of stuck where they are currently at in the rankings with the highest tier univerisites only moving a few spots each year. The schools at the top of these lists are often the most historical schools with the most demand from prospective students. The demand leads to a competitive class year after year for these schools. The narrow path of these rankings leads to these historically competitive research insitutions value selectivity and limiting the number of students into the program.  I believe different metrics that are more inclusive of public in-state and growing universities such as UNC Charlotte can be used. In the metrics chosen UNC Charlotte performs poorly when compared to its rankings in the other critiera. A good portion of students are part time students. The identity of UNC Charlotte is also not currently to become extremely selective or limit the time in which people are allowed to graduate in order to increase these rankings. 

An obvious metric for traditional national rankings is the entrance profile of admitted students. Factors including GPA, test scores, and extracurriculars are scrutinized and are up to the interpretation of admissions committees on whether they could be admitted into the program. Every year there are many students that apply to high ranking universities that very well could succeed in the program but are denied due to head counts and limiting seats in order to limit faculty to student ratio and other related metrics. This selectivity increases the prowess of the school attracting highly intelligent students to apply for these selective colleges for the chance of admission and increase their chances of being successful in their career in academia and industry. Historically important and older institutions are often viewed as having a higher quality of education with many wanting to go there and willing to even pay out-of-state tuition in order for them to attend a particular school. These older universities also have important alumni from the past and present that advocate and draw research funding and endowments to a research institution and that is extremely valuable. Programs with more funding are often on the cutting edge of research and have relationships with the most valuable companies in the world to partner and prepare students for work in their company. 


part 2

In [52]:
import pandas as pd
import numpy as np

colleges = pd.read_csv('/Users/jgmac22/Downloads/colleges.csv', low_memory=False)

# Filter colleges with undergraduate populations greater than 700
colleges = colleges[colleges['UGDS'] > 700]

# Replace 'PrivacySuppressed' with NaN and convert to numeric for several columns
columns_to_convert = [
    'ADM_RATE_SUPP', 'C150_4_POOLED_SUPP', 'MN_EARN_WNE_P10', 'STUFACR', 
    'RET_FT4_POOLED_SUPP', 'SAT_AVG_ALL', 'NPT4_PUB', 'ACTCM75', 
    'PCTPELL_DCS_POOLED_SUPP', 'PPLUS_PCT_LOW_POOLED_SUPP', 'GRAD_DEBT_MDN_SUPP'
]
for col in columns_to_convert:
    colleges[col] = colleges[col].replace('PrivacySuppressed', np.nan)
    colleges[col] = pd.to_numeric(colleges[col], errors='coerce')

# Add the earnings to debt ratio
colleges = colleges.assign(earnings_debt_ratio=(colleges["MN_EARN_WNE_P10"]/colleges["GRAD_DEBT_MDN_SUPP"]))

# Drop rows where any of the key metrics is NaN
colleges = colleges.dropna(subset=columns_to_convert + ['earnings_debt_ratio'])

# Rankings based on various metrics including the new earnings to debt ratio
colleges['Selectivity Rank'] = colleges['ADM_RATE_SUPP'].rank(method='min', ascending=True)
colleges['Graduation Rate Rank'] = colleges['C150_4_POOLED_SUPP'].rank(method='min', ascending=False)
colleges['Earnings Rank'] = colleges['MN_EARN_WNE_P10'].rank(method='min', ascending=False)
colleges['Faculty Student Rank'] = colleges['STUFACR'].rank(method='min', ascending=True)
colleges['Retention Rate Rank'] = colleges['RET_FT4_POOLED_SUPP'].rank(method='min', ascending=False)
colleges['SAT Score Rank'] = colleges['SAT_AVG_ALL'].rank(method='min', ascending=False)
colleges['Net Price Rank'] = colleges['NPT4_PUB'].rank(method='min', ascending=True)
colleges['ACT Score Rank'] = colleges['ACTCM75'].rank(method='min', ascending=False)
colleges['Pell Grant Percentage Rank'] = colleges['PCTPELL_DCS_POOLED_SUPP'].rank(method='min', ascending=False)
colleges['Parent PLUS Loan Percentage Rank'] = colleges['PPLUS_PCT_LOW_POOLED_SUPP'].rank(method='min', ascending=True)
colleges['Earnings Debt Ratio Rank'] = colleges['earnings_debt_ratio'].rank(method='min', ascending=False)  

colleges['Combined Rank'] = colleges[[
    'Selectivity Rank', 'Graduation Rate Rank', 'Earnings Rank', 
    'Faculty Student Rank', 'Retention Rate Rank', 'SAT Score Rank', 
    'Net Price Rank', 'ACT Score Rank', 'Pell Grant Percentage Rank', 
    'Parent PLUS Loan Percentage Rank', 'Earnings Debt Ratio Rank'
]].mean(axis=1)

# Sort colleges by combined rank
ranked_colleges = colleges[[
    'INSTNM', 'ADM_RATE_SUPP', 'C150_4_POOLED_SUPP', 'MN_EARN_WNE_P10', 'STUFACR', 
    'RET_FT4_POOLED_SUPP', 'SAT_AVG_ALL', 'NPT4_PUB', 'ACTCM75', 'PCTPELL_DCS_POOLED_SUPP', 
    'PPLUS_PCT_LOW_POOLED_SUPP', 'GRAD_DEBT_MDN_SUPP', 'earnings_debt_ratio', 'Selectivity Rank', 
    'Graduation Rate Rank', 'Earnings Rank', 'Faculty Student Rank', 'Retention Rate Rank', 
    'SAT Score Rank', 'Net Price Rank', 'ACT Score Rank', 'Pell Grant Percentage Rank', 
    'Parent PLUS Loan Percentage Rank', 'Earnings Debt Ratio Rank', 'Combined Rank'
]]
ranked_colleges_sorted = ranked_colleges.sort_values(by='Combined Rank')
ranked_colleges_sorted.reset_index(drop=True, inplace=True)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

print(ranked_colleges_sorted.head(50))  # Shows the top 50 entries based on the combined rank


                                               INSTNM  ADM_RATE_SUPP  \
0                               University of Florida         0.2335   
1                    University of Michigan-Ann Arbor         0.1769   
2         University of North Carolina at Chapel Hill         0.1711   
3               United States Merchant Marine Academy         0.2735   
4                  New Jersey Institute of Technology         0.6599   
5                            Florida State University         0.2504   
6                                      William & Mary         0.3350   
7                       Purdue University-Main Campus         0.5270   
8                           Rutgers University-Camden         0.7652   
9                  University of Virginia-Main Campus         0.1866   
10                       University of Florida-Online         0.6464   
11                             Stony Brook University         0.4914   
12         North Carolina State University at Raleigh         0.

In my new ranking system, I decided to use the ranking of different factors I believe are most important to the school. Each metric was evaluated with a lower ranking being favorable for each category. The sum of each category for each school was calculated to rank the schools with number one having the lowest sum of scores across all categories. I had the goal of making a ranking system that is more inclusive and realistic to the average prospective student. The new ranking systems takes into account financial aid, cost of attendance, student loan delinquency rates, and proportion of students that are receiving a pell grant. The traditional ranking categories are also included but each category is viewed equally and the sum for each school is ranked in the combined rank column. These are extrememly important factors to most students that arent neccesariely weighted or viewed equally when compared to traditional ranking systems. Private universities are extremely expensive for out of state students and without grants or a scholarship the proscpect of being able to realistically enter those programs is diminished. This is an unfortunate reality of these extremely selective schools in the current climate of tuition prices across the country. One important metric that I cant locate is research funding availability for students. This is one of the most important factors that students should consider when choosing where pursue their education. The highest ranking universities have millions of dollars in reserch funding available to pursue the must cutting edge and important topics seen in every field. Faculty and staff need the capability to access expensive cutting edge technologies in order to make accurate and respectable research conclusions. This is one metric I believe should be further researched to be included in these rankings. It likely is used to some extent by the publishers but the data is not publically available to my knowledge. The following categories were used in my analysis and rankign assignemnts. 


Selectivity Rank (ADM_RATE_SUPP): Ranks colleges based on their acceptance rate, where a lower rate (more selective) results in a higher rank.
Graduation Rate Rank (C150_4_POOLED_SUPP): Ranks colleges by graduation rates, with higher rates resulting in better ranks.
Earnings Rank (MN_EARN_WNE_P10): Ranks colleges by median earnings of former students, with higher earnings leading to a higher rank.
Faculty Student Rank (STUFACR): Ranks colleges by their faculty-to-student ratio, where a lower ratio (more faculty per student) results in a better rank.
Retention Rate Rank (RET_FT4_POOLED_SUPP): Ranks colleges by student retention rates, with higher rates indicating better performance.
SAT Score Rank (SAT_AVG_ALL): Ranks colleges by average SAT scores of admitted students, with higher scores leading to better ranks.
Net Price Rank (NPT4_PUB): Ranks public colleges by their net price, with lower prices resulting in better ranks.
ACT Score Rank (ACTCM75): Ranks colleges by the 75th percentile ACT scores, where higher scores result in better ranks.
Pell Grant Percentage Rank (PCTPELL_DCS_POOLED_SUPP): Ranks colleges by the percentage of students receiving Pell Grants, with higher percentages leading to better ranks.
Parent PLUS Loan Percentage Rank (PPLUS_PCT_LOW_POOLED_SUPP): Ranks colleges by the percentage of parents taking low-income PLUS loans, with higher percentages resulting in better ranks.
Earnings Debt Ratio Rank (earnings_debt_ratio): Ranks colleges by the ratio of median earnings to median graduate debt, with higher ratios indicating better financial outcomes for graduates.


part 3

I chose to look at bioinformatics programs and use my two rankings systems to compare them based on a specific field of study. In the field of study analysis performed by the Department of Education; Bioinformatics, Biomathematics, and Computational Biology the data is very limited so I decided to include general biology, biotechnology,bioengineering,  biophysics, and physical chemistry in my new ranking system. This will allow for a far more amount of data to be used in the ranking system. Its dissapointing that bioinformatics cannot be looked at specifically but that is the nature of the public data releases with withheld information. The metrics used in step one of this analysis are not available so the closest metric was used instead in order to compare school rankings when looking only in biological sciences and bioinformatics. It was difficult for me to find equivalent metrics given so little data was included in the field of study csv file. I tried to make a different methodology for the below rankings but they ended up being closer to one another than different. 

In [48]:
import pandas as pd

# Load the Field of Study data
collegesFOS = pd.read_csv('/Users/jgmac22/Downloads/Most-Recent-Cohorts-Field-of-Study.csv', low_memory=False)

# Filter the Field of Study data for the specific CIP codes
bioinformatics_df = collegesFOS[collegesFOS['CIPCODE'].isin([2611, 2612, 2613, 2602, 2601,1405,2699])]

# Prepare the data by replacing 'PrivacySuppressed' and converting to numeric for ranking
columns_to_convert = ['EARN_MDN_4YR']  # Focusing only on earnings data
for col in columns_to_convert:
    bioinformatics_df.loc[:, col] = bioinformatics_df[col].replace('PS', np.nan)  # Assuming 'PS' stands for 'PrivacySuppressed'
    bioinformatics_df.loc[:, col] = pd.to_numeric(bioinformatics_df[col], errors='coerce')

# Ensure no necessary data is missing before ranking
bioinformatics_df.dropna(subset=columns_to_convert, inplace=True)

# Apply ranking for earnings
bioinformatics_df['Earnings 4yr Median Rank'] = bioinformatics_df['EARN_MDN_4YR'].rank(method='min', ascending=False)

# Sort by earnings rank for a better overview
bioinformatics_df_sorted = bioinformatics_df.sort_values(by='Earnings 4yr Median Rank')
bioinformatics_df_sorted.reset_index(drop=True, inplace=True)

pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', None)     # Show all rows, adjust based on actual data size

# Display results, now showing the sorted DataFrame focusing only on earnings
print(bioinformatics_df_sorted[['INSTNM', 'EARN_MDN_4YR', 'Earnings 4yr Median Rank']].head(500))


                                                INSTNM EARN_MDN_4YR  \
0                    Ohio State University-Main Campus     253781.0   
1                    University of Wisconsin-La Crosse     181793.0   
2              University of Washington-Seattle Campus     177110.0   
3                           University of Pennsylvania     173240.0   
4                                   Harvard University     171755.0   
5          Columbia University in the City of New York     138598.0   
6                   University of California-San Diego     131462.0   
7                                     Tufts University     131012.0   
8                             Johns Hopkins University     130224.0   
9                                      Duke University     125061.0   
10                                  Harvard University     122776.0   
11                                  Cornell University     119329.0   
12               Massachusetts Institute of Technology     116182.0   
13    

/var/folders/yw/73c0bp3s12l0_zbfbgfn2d580000gn/T/ipykernel_69389/2228158011.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bioinformatics_df.dropna(subset=columns_to_convert, inplace=True)
/var/folders/yw/73c0bp3s12l0_zbfbgfn2d580000gn/T/ipykernel_69389/2228158011.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bioinformatics_df['Earnings 4yr Median Rank'] = bioinformatics_df['EARN_MDN_4YR'].rank(method='min', ascending=False)


In [47]:
import pandas as pd

# Load the Field of Study data
collegesFOS = pd.read_csv('/Users/jgmac22/Downloads/Most-Recent-Cohorts-Field-of-Study.csv', low_memory=False)

# Filter the Field of Study data for the specific CIP codes 2611, 2612, 2613, 2602, and 2601
bioinformatics_df = collegesFOS[collegesFOS['CIPCODE'].isin([2611, 2612, 2613, 2602, 2601,1405,2699])]

# Prepare the data by replacing 'PrivacySuppressed' and converting to numeric for ranking
columns_to_convert = ['DEBT_ALL_STGP_EVAL_MDN', 'EARN_MDN_4YR', 'BBRR4_FED_COMP_DLNQ', 'BBRR1_FED_COMP_PAIDINFULL']
for col in columns_to_convert:
    bioinformatics_df.loc[:, col] = bioinformatics_df[col].replace('PS', np.nan)  # Assuming 'PS' stands for 'PrivacySuppressed'
    bioinformatics_df.loc[:, col] = pd.to_numeric(bioinformatics_df[col], errors='coerce')

# Ensure no necessary data is missing before ranking
bioinformatics_df.dropna(subset=columns_to_convert, inplace=True)

# Apply rankings separately for each category within the filtered group using .loc to specify the DataFrame
bioinformatics_df['Median Debt Rank'] = bioinformatics_df['DEBT_ALL_STGP_EVAL_MDN'].rank(method='min', ascending=True)
bioinformatics_df['Earnings 4yr Median Rank'] = bioinformatics_df['EARN_MDN_4YR'].rank(method='min', ascending=False)
bioinformatics_df['Delinquency Rate Rank'] = bioinformatics_df['BBRR4_FED_COMP_DLNQ'].rank(method='min', ascending=True)
bioinformatics_df['Full Payment Rate Rank'] = bioinformatics_df['BBRR1_FED_COMP_PAIDINFULL'].rank(method='min', ascending=False)

# Calculate the combined rank as an average of the individual ranks
bioinformatics_df['Combined Rank'] = bioinformatics_df[['Median Debt Rank', 'Earnings 4yr Median Rank', 'Delinquency Rate Rank', 'Full Payment Rate Rank']].mean(axis=1)

# Sort by combined rank for a better overview
bioinformatics_df_sorted = bioinformatics_df.sort_values(by='Combined Rank')
bioinformatics_df_sorted.reset_index(drop=True, inplace=True)

pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', None)     # Show all rows, adjust based on actual data size

# Display results, now showing the sorted DataFrame
print(bioinformatics_df_sorted[['INSTNM', 'DEBT_ALL_STGP_EVAL_MDN', 'Median Debt Rank', 'EARN_MDN_4YR', 'Earnings 4yr Median Rank', 'Combined Rank']].head(500))


                                                INSTNM DEBT_ALL_STGP_EVAL_MDN  \
0                California State University-Fullerton                18240.0   
1                               Utah Valley University                17015.0   
2                              Winona State University                21500.0   
3                       San Francisco State University                15000.0   
4               California State University-Northridge                15000.0   
5                  University of Minnesota-Twin Cities                19500.0   
6               California State University-Stanislaus                14116.0   
7            Arizona State University Campus Immersion                21000.0   
8                       University of Illinois Chicago                14723.0   
9                     Northeastern Illinois University                13075.0   
10                 University of Wisconsin-Platteville                20467.0   
11                          

/var/folders/yw/73c0bp3s12l0_zbfbgfn2d580000gn/T/ipykernel_69389/547831541.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bioinformatics_df.dropna(subset=columns_to_convert, inplace=True)
/var/folders/yw/73c0bp3s12l0_zbfbgfn2d580000gn/T/ipykernel_69389/547831541.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bioinformatics_df['Median Debt Rank'] = bioinformatics_df['DEBT_ALL_STGP_EVAL_MDN'].rank(method='min', ascending=True)
/var/folders/yw/73c0bp3s12l0_zbfbgfn2d580000gn/T/ipykernel_69389/547831541.py:20: SettingWithCopyWarning: 
A value is trying to 

part 4

One area that boosts UNC Charlottes ranking compared to other schools its its relatively lower cost of attendance compared to the quality of education you receive. Many of the highest ranking schools in The United States are private and or extremely expensive for the average american household unless need based grants or scholarships are offered to the student. Most students are limited in scope to in state public universities that they can afford. UNC Charlotte could possibly further decrease the cost of tuition to further this ranking but could lead to affect in other areas such as student life and involvement. In the national rankings UNC CHarlotte was in the following rankings. Becoming more selective in admissions is also one way to boost the rankings overall:
US News - 178
Forbes - 362
my rank
These are obviously extrmemly different rankings and is interesting to see the degree of difference. They most likely have very different weightings of various metrics leading to the differnece. When financials are taken out of the  analaysis the picture becomes much closer to the tradtional rankings. This is because the high cost of some insitutions is affecting the rankings in the revised scoring scheme. More work can be done in the future to clean up the rankings by further gathering data on bioinformatics and related disciplines. 

In the traditional ranking system I attempted to re-model UNC Charlotte and it was ranked 382. In my new ranking system it was rated 118. Both of these are relatively similar to the national rankings and demonstrates how these ranking are somehwat calculated. One trend I see in the traditional ranking system is the faculty-to-student ratio for higher-ranked schools is much lower with more faculty per student. For example, only 28% of classes at UNC Charlotte have fewer than 20 students. Compared to top-ranked Princeton from US News with 74% of classes having less than 20 students (US News). This trend is also perpetuated by how selective and small proportion of applicants they accept into their programs. Although this is an important metric for choosing a school, much emphasis is placed on the number of students in a class when judging a school based on the quality of education it offers. Another metric that seems to be important based upon the emphasis in the ranking reports is the 4 year graduation rate. 98 % of students graduate within 6 years of starting school at Princeton. At UNC Charlotte most of our students dont graduate within the 4 year timeline as many are part time students that live and work around the city (. Although this does not necessarily affect the quality of education they receive, it affects the national rankings of the university. It would be difficult for UNC Charlotte to want to try and increase this rate because it goes against the identity of the institution. This is a university in a large metro area and provides valuable resources to working professionals who also have the purchasing power to afford part time school. This negatively affects the rankings of the school but positively boosts tuition received from these students being able to attend while working a professional job during normal business hours. 


